# Objectives:
1) predict the chances of surviving in a catastrophic event like titanic.

# Getting DATA

In [21]:
# #GEtting Titanic dataset from kaggle competition
# import os
# from kaggle.api.kaggle_api_extended import KaggleApi
# import zipfile

# # #setup the kaggle API key 
# os.environ['KAGGLE_CONFIG_DIR'] = r"C:\Users\haide\OneDrive\Desktop\api's\kaggle.json"
# api = KaggleApi()
# api.authenticate()

# # #competition dataset identifier
# competition_name = 'titanic'
# path='./titanic_dataset'
# # #Fetch dataset from the Titanic Competition
# api.competition_download_files(competition_name,path=path)

# # Unzip the downloaded files manually
# with zipfile.ZipFile(os.path.join(path, 'titanic.zip'), 'r') as zip_ref:
#     zip_ref.extractall(path)


# print("DONE")


In [162]:
from sklearn.model_selection import train_test_split
import pandas as pd

X = pd.read_csv('./titanic_dataset/titanic_train.csv')
y = pd.DataFrame(X['survived'])
X = X.drop(['survived'],axis=1)
X_test = pd.read_csv('./titanic_dataset/titanic_test.csv')
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

In [163]:
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (680, 14)
y_train:  (680, 1)
X_val:  (170, 14)
y_val:  (170, 1)
X_test:  (459, 14)


# Data Analysis and Preprocessing

In [164]:
# checking features usablility and filtering out the good one's
X_train.info()
X_train['name'].describe() #rremove
#name feature is unique for each instance hence is not useful
X_train['ticket'].describe() #remoce
#ticket id isn't useful cause it's unique for almost instance
X_train['cabin'].head() #frequency encoding and remove
# cabin id can be used with some preprocessing(we can know which deck side had more chances of surviving)
X_train['embarked'].describe() #frequency encoding and remove
# embarked can encoded and stored
X_train['boat'].describe() #frequency encoding and remove
#only 242 people were issued boat at evacuation. it will be useful.
X_train['body'].describe() 
# only 60 people body were found. we will know which bodies(not surviving one's) were found with what similarities.
X_train['home.dest'].describe() #frequency encoding and remove
#we can use home.dest to keep track of allocations or lifestyle and some patterns between people
X_train['sex'].describe()
#useful
X_train['age'].describe()
#useful
X_train['sibsp'].describe()
#useful
X_train['parch'].describe()
#well almost all values in parch are 0. but let's keep it for now
X_train['fare'].describe()
#useful for patterns of variation within classes.
X_train['pclass'].describe()
#useful

#removing  cols name and ticket
X_train = X_train.drop(['name','ticket'],axis=1)
X_val = X_val.drop(['name','ticket'],axis=1)
X_test = X_test.drop(['name','ticket'],axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, 332 to 102
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  680 non-null    int64  
 1   pclass        680 non-null    int64  
 2   name          680 non-null    object 
 3   sex           680 non-null    object 
 4   age           539 non-null    float64
 5   sibsp         680 non-null    int64  
 6   parch         680 non-null    int64  
 7   ticket        680 non-null    object 
 8   fare          679 non-null    float64
 9   cabin         155 non-null    object 
 10  embarked      680 non-null    object 
 11  boat          242 non-null    object 
 12  body          60 non-null     float64
 13  home.dest     379 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 79.7+ KB


In [165]:
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (680, 12)
y_train:  (680, 1)
X_val:  (170, 12)
y_val:  (170, 1)
X_test:  (459, 12)


In [166]:
# Handling missing values

num_cols = X_train.select_dtypes(include=['int64','float64']).columns
cat_cols = X_train.select_dtypes(include=['object']).columns



from sklearn.impute import SimpleImputer

#for cat_cols
mode_imputer = SimpleImputer(strategy='most_frequent')
X_train[cat_cols] =  mode_imputer.fit_transform(X_train[cat_cols])
X_val[cat_cols] = mode_imputer.transform(X_val[cat_cols])
X_test[cat_cols] = mode_imputer.transform(X_test[cat_cols])

#for num_cols
mean_imputer = SimpleImputer(strategy='mean')
X_train[num_cols] = mean_imputer.fit_transform(X_train[num_cols])
X_val[num_cols] = mean_imputer.transform(X_val[num_cols])
X_test[num_cols] = mean_imputer.transform(X_test[num_cols])


In [167]:
#check for missing values
print(X_train[cat_cols].isnull().sum())
print(X_test[cat_cols].isna().sum())
print(X_val[cat_cols].isna().sum())


sex          0
cabin        0
embarked     0
boat         0
home.dest    0
dtype: int64
sex          0
cabin        0
embarked     0
boat         0
home.dest    0
dtype: int64
sex          0
cabin        0
embarked     0
boat         0
home.dest    0
dtype: int64


In [168]:
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (680, 12)
y_train:  (680, 1)
X_val:  (170, 12)
y_val:  (170, 1)
X_test:  (459, 12)


In [169]:
#Encoding cat cols:
import numpy as np

for col in cat_cols:
    mapping = X_train[col].value_counts()
    
    # Apply mapping to train, test, and validation sets
    X_train[col + '_encoded'] = X_train[col].map(mapping).fillna(-1)  # Default missing categories to -1
    X_test[col + '_encoded'] = X_test[col].map(mapping).fillna(-1)    # Default missing categories to -1
    X_val[col + '_encoded'] = X_val[col].map(mapping).fillna(-1)      # Default missing categories to -1



#ohe
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
train_encoded_cols = encoder.fit_transform(X_train[['sex']])
test_encoded_cols = encoder.transform(X_test[['sex']])
val_encoded_cols = encoder.transform(X_val[['sex']])

X_train_ohe = pd.DataFrame(train_encoded_cols,columns=encoder.get_feature_names_out(['sex']))
X_test_ohe = pd.DataFrame(test_encoded_cols,columns=encoder.get_feature_names_out(['sex']))
X_val_ohe = pd.DataFrame(val_encoded_cols,columns=encoder.get_feature_names_out(['sex']))

X_train = pd.concat([X_train,X_train_ohe],axis=1)
X_test = pd.concat([X_test,X_test_ohe],axis=1)
X_val = pd.concat([X_val,X_val_ohe],axis=1)



In [181]:
#imputing the sex ohe cols
mean_imputer = SimpleImputer(strategy='mean')
X_train[['sex_female','sex_male']] = mean_imputer.fit_transform(X_train[['sex_female','sex_male']])
X_val[['sex_female','sex_male']] = mean_imputer.transform(X_val[['sex_female','sex_male']])
X_test[['sex_female','sex_male']] = mean_imputer.transform(X_test[['sex_female','sex_male']])


In [170]:
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (810, 19)
y_train:  (680, 1)
X_val:  (307, 19)
y_val:  (170, 1)
X_test:  (459, 19)


In [171]:
#check for missing values
print(X_train[cat_cols].isnull().sum())
print(X_test[cat_cols].isna().sum())
print(X_val[cat_cols].isna().sum())

sex          130
cabin        130
embarked     130
boat         130
home.dest    130
dtype: int64
sex          0
cabin        0
embarked     0
boat         0
home.dest    0
dtype: int64
sex          137
cabin        137
embarked     137
boat         137
home.dest    137
dtype: int64


In [172]:
# Drop rows with NaN values only in categorical columns
X_train = X_train.dropna(subset=cat_cols)
X_test = X_test.dropna(subset=cat_cols)
X_val = X_val.dropna(subset=cat_cols)


#check dims
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (680, 19)
y_train:  (680, 1)
X_val:  (170, 19)
y_val:  (170, 1)
X_test:  (459, 19)


In [173]:
#drop original cols
X_train = X_train.drop(cat_cols,axis=1)
X_test = X_test.drop(cat_cols,axis=1)
X_val = X_val.drop(cat_cols,axis=1)


In [174]:
#check dims
print('X_train: ',X_train.shape)
print('y_train: ',y_train.shape)
print("X_val: ",X_val.shape)
print("y_val: ",y_val.shape)
print("X_test: ",X_test.shape)

X_train:  (680, 14)
y_train:  (680, 1)
X_val:  (170, 14)
y_val:  (170, 1)
X_test:  (459, 14)


In [175]:
#check dimensions
y_train = y_train.to_numpy().ravel()
X_train.shape,y_train.shape

((680, 14), (680,))

# Training

In [183]:
from sklearn.neighbors import KNeighborsClassifier
knn1 = KNeighborsClassifier().fit(X_train,y_train)

# Evaluation

In [184]:
preds = knn1.predict(X_train)
preds

from sklearn.metrics import precision_score,recall_score,confusion_matrix,accuracy_score
from sklearn.model_selection import cross_val_predict
y_train_preds = cross_val_predict(knn1,X_train,y_train,cv=3)

In [185]:
print('precision_score: ',precision_score(y_train,y_train_preds))
print('recall score: ',recall_score(y_train,y_train_preds))
print('confusion matrix: ',confusion_matrix(y_train,y_train_preds))

precision_score:  0.9279661016949152
recall score:  0.8938775510204081
confusion matrix:  [[418  17]
 [ 26 219]]


# Insights:
1) model prediction are bad. let's try finetuning using gridsearchcv 

# FineTuning

In [186]:
from sklearn.model_selection import GridSearchCV

param_grid={
    'n_neighbors':[3,5,7,10],
    'weights': ['uniform','distance'],
    'metric': ['euclidean','manhattan']
}

knn2  =KNeighborsClassifier()
grid_search = GridSearchCV(knn2,param_grid,cv=3,
                           verbose=1,n_jobs=-1)


grid_search.fit(X_train,y_train)

print("Best Parameters: ", grid_search.best_params_)
print("Best Estimators: ", grid_search.best_estimator_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Best Parameters:  {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
Best Estimators:  KNeighborsClassifier(metric='euclidean', n_neighbors=3)


In [187]:
knn3  = grid_search.best_estimator_
knn3preds = knn3.predict(X_train)

print('accuracy_score: ', accuracy_score(y_train,knn3preds))


accuracy_score:  0.9632352941176471


In [188]:
# comparing with a dumb model that predicts just 0
y_train_no_preds = np.zeros_like(y_train)


print('accuracy_score: ',accuracy_score(y_train,y_train_no_preds))

accuracy_score:  0.6397058823529411


In [190]:
# score on val set
knn3  = grid_search.best_estimator_
knn3preds = knn3.predict(X_val)

print('accuracy_score: ', accuracy_score(y_val,knn3preds))


accuracy_score:  0.9823529411764705


In [191]:
# preds of test set
knn3  = grid_search.best_estimator_
knn3preds_test = knn3.predict(X_test)

# Conclusion
1) Our model is performing much better than a base line model
2) best model knn3 with 97-98% accuracy

In [194]:
import pandas as pd

def create_kaggle_submission(test_data, predictions, output_filename='submission.csv'):
    """
    Generates a Kaggle submission CSV file.
    
    Parameters:
    test_data (DataFrame): The test data that contains the 'Id' column.
    predictions (array-like): The predicted labels or values.
    output_filename (str): The name of the output CSV file (default is 'submission.csv').
    
    Returns:
    None: Saves the submission CSV file in the current directory.
    """
    # Create a DataFrame with the 'Id' column and predictions
    submission_df = pd.DataFrame({
        'passenger_id': test_data['passenger_id'].astype(np.int32),  # Ensure your test data has an 'Id' column
        'Survived': predictions  # Replace with the correct name if needed
    })
    
    # Save the DataFrame to a CSV file
    submission_df.to_csv(output_filename, index=False)
    print(f"Submission file '{output_filename}' created successfully!")

# Example usage:
# Assuming you have the test data (test_data) and predictions (y_pred)
# create_kaggle_submission(test_data, y_pred)
create_kaggle_submission(X_test,knn3preds_test)


Submission file 'submission.csv' created successfully!


# Conclusion
1) The score on test set was 0.962.